## Getting Started with Batfish

This notebook uses pybatfish, a Python-based SDK for Batfish, to analyze a sample network. It shows how to submit your configurations and other network data for analysis and how to extract information from its vendor neutral data model. 

### Initializing a Network and Snapshot

A *network* is a logical group of routers and links. It can be your entire network or a subset of it. A *snapshot* is a collection of information (configuration files, routing data, etc…) that represent the network state at a point in time. Snapshots can contain the actual configuration of network devices or candidate configurations.

**NB: Make sure that the Batfish service is running on localhost before proceeding further**

In [ ]:
# Import pybatfish and other needed packages
%run startup.py

In [ ]:
# Assign a friendly name to your network and snapshot
NETWORK_NAME = "example_network"
SNAPSHOT_NAME = "example_snapshot"

# Update SNAPSHOT_PATH to point to a directory containing your network snapshots.
# Example snapshots are available in the test_rigs folder of the Batfish repository.
# See [here](https://github.com/batfish/batfish/wiki/Packaging-testrigs-for-analysis) for instructions on packaging your own data for analysis.
SNAPSHOT_PATH = "../../batfish/test_rigs/example"

# Now create the network and initialize the snapshot
bf_set_network(NETWORK_NAME)
bf_init_snapshot(SNAPSHOT_PATH, name=SNAPSHOT_NAME, overwrite=True)

### Extracting Information
Batfish creates a comprehensive vendor neutral device and network model from which information such as list of devices, interface state, VRFs etc. can be extracted and validated. This notebook focuses on extractions. See other notebooks on how to validate extracted information. 

*Questions* are the mechanism by which you extract information from the Batfish service about your network and snapshot(s).

In [ ]:
# Load questions from Batfish
load_questions()

In [ ]:
# To see available questions, you can use
list_questions()

# You can also use tab-completion on the Batfish question module - bfq. -> press TAB key,
# uncomment and try on the following line
# bfq.

# You can find out usage for a question by calling .help() on it; note the `()` before .help()
bfq.nodeProperties().help()

### Getting status of parsed files

Batfish may ignore certain lines in the configuration. To retrieve the parsing status of snapshot files, use the fileParseStatus question

In [ ]:
parse_status = bfq.fileParseStatus().answer().frame()

`answer()` runs the question and returns the answer in a JSON format. 

`frame()` wraps the answer as [pandas dataframe](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html).

Additional post-processing can be done on this data, like filtering for values in one or multiple columns, reducing the number of columns, etc. using pandas. This post-processing can be the basis of lightweight validation; Batfish also includes a range of questions for deeper validation (as discussed in other notebooks).

Information on pandas-based processing can be found in the [pandas tutorial on filtering](http://nbviewer.jupyter.org/github/jvns/pandas-cookbook/blob/v0.2/cookbook/Chapter%203%20-%20Which%20borough%20has%20the%20most%20noise%20complaints%20%28or%2C%20more%20selecting%20data%29.ipynb).

In [ ]:
# An example: use a filter on the returned dataframe to see which files failed to parse completely
parse_status[parse_status['Status'] != 'PASSED']  # change '!=' to '==' to get the files which passed

In [ ]:
# View details if some of the files were not parsed completely
parse_warning = bfq.parseWarning().answer().frame()

parse_warning

### Inspecting referential integrity of configuration structures
Network configuratons define and reference named structures like route maps, access control lists (ACLs), prefix lists, etc. Two common indicators of buggy configurations include references to structures that are not defined anywhere or defined structures that are not referenced anywhere. Batfish makes it easy to flag such instances because it understand the underlying semantics of configuration.

##### TODO: cells for undefined references and unused structures

### Extracting properties of network entities
Entities in the network refer to things like nodes, interfaces, routing processes, and VRFs. Batfish makes it trivial to extract configured properties of such entities in a vendor neutral manner. 

##### Node properties
The nodeProperties question extract information on nodes in the snapshot.

In [ ]:
# Extract the properties of all nodes whose names match the regular expression '.*border.*'
node_properties = bfq.nodeProperties(nodeRegex=".*border.*").answer().frame()

In [ ]:
# View what columns (properties) are present in the answer
node_properties.columns

In [ ]:
# To extract only a subset of properties, use the propertySpec parameter
node_properties_trunc = bfq.nodeProperties(nodeRegex=".*border.*", propertySpec="domain-name|ntp-servers|interfaces").answer().frame()

node_properties_trunc

An alternative (client-side) way to restrict the list of columns displayed is to use pandas-based column filtering ([pandas tutorial](http://nbviewer.jupyter.org/github/jvns/pandas-cookbook/blob/v0.2/cookbook/Chapter%202%20-%20Selecting%20data%20%26%20finding%20the%20most%20common%20complaint%20type.ipynb)).

In [ ]:
# Let's remove the interfaces column from our result
node_properties_trunc = node_properties_trunc[["node", "domain-name", "ntp-servers"]]

node_properties_trunc

You can add additional filters to restrict entries based on values of columns.

In [ ]:
# View only nodes with **23.23.23.23** as one of the configured ntp-servers
node_properties_trunc[node_properties_trunc['ntp-servers'].apply(lambda x:'23.23.23.23' in x)]

#### Interface properties
To retrieve information about interfaces present and the properties of them, use the **interfaceProperties** question

In [ ]:
interface_properties = bfq.interfaceProperties(nodeRegex=".*border.*", propertySpec="interface-type|bandwidth|vrf|primary-address").answer().frame()

If you wanted to just find interfaces with the primary ip address in <b>10.12.0.0/16</b>, you can filter the results as shown below.

**na=False** is required in order to ignore interfaces without any configured IP addresses, such as ethernet switchports.


In [ ]:
interface_properties[interface_properties['primary-address'].str.match("10.12", na=False)]

Similar questions extract properties of other entities (e.g., bgpProperties() extracts properties of BGP processes).

### Exploring Routing and Forwarding Tables (RIBs and FIBs)
Batfish can compute routing and forwarding tables (aka dataplane) of the network from snapshot data itself. This data plane can be examined to understand the routing and forwarding behavior of the network. 

In [ ]:
# Fetch the routing table of all VRFs on all nodes in the snapshot
routes_df = bfq.routes().answer().frame()

(For a large network, the first time you run a question that needs the dataplane, fetching the answer can take a few minutes. Subsequent questions are quick as the generated dataplane is saved by Batfish.)

As used above, the routes() question can generate a lot of results. You may restrict the output by using filter in the question. To restrict the results to **border** routers, use **nodeRegex = "\.\*border\.\*"** as parameters for the routes question. 

As for properties, you can also just filter the results on the client side using pandas. For example, if you wanted to see all the routes on all the nodes/VRFs for the network **90.90.90.0/24** with an **Admin Distance of 0**", you can filter using multiple conditions in [pandas](http://pandas.pydata.org/pandas-docs/version/0.15/indexing.html#boolean-indexing)

In [ ]:
routes_df[(routes_df['Network'] == "90.90.90.0/24") & (routes_df["AdminDistance"] == 0)]

***
That's it for now! Feel free to explore further by adding cells and running other questions, or play with the other notebooks in the repository.